In [1]:
import beaapi
import numpy as np
import pandas as pd

In [2]:
# Get key from unversioned file
import os
from dotenv import load_dotenv
load_dotenv()
beaKey = os.environ.get("beakey") #and be careful as USERID is passed back in the 'param' dict of many return values.

# Reshaping tables from each directorate

Look at examples tables from each directorate. Try to determine what is the unique index for each table. Therefore ask for tables with multiple values for each query parameter when possible. Usually we can do this with 1 table spec from a directorate, but sometimes we need two. 

Basic reshaping work done. Left to do:
- Maybe Return time_variant_only_vars data?
- When resulting data isn't even panel, show how to split tables (typically by frequency so that it is)
- Maybe drop DataFormatted? (See how they differ, check docs) 

In [3]:


def show_tables(dataset, spec, reraise=True, check_note_linkage=True):
    tbls = [None, None, None]
    try:
        bea_tbl = beaapi.get_data(beaKey, dataset, **spec)
        #display(bea_tbl.dtypes)

        tbls[0] = bea_tbl.set_index(bea_tbl.attrs['index_cols'])
        display(tbls[0].head(1))
        #print(bea_tbl.attrs)

        tbls[1] = beaapi.to_wide_vars_in_cols(bea_tbl)        
        display(tbls[1].head(1))
        
        tbls[2] = beaapi.to_wide_vars_in_rows(bea_tbl)
        display(tbls[2].head(1))

    except Exception as e:
        if reraise:
            raise e
        print(e)
    
    if check_note_linkage and 'NoteRef' in tbls[0].attrs['detail']:
        ref = tbls[0].attrs['detail']['NoteRef']
        valid_note_ref = ref in list(tbls[0].attrs['detail']['Notes']['NoteRef'])
        if not valid_note_ref:
            msg = "Linkage Error: Result NoteRef " + ref + " not found in Notes nodes."
            if reraise:
                raise Exception(msg)
            else:
                print(msg)
    
    return(tbls)

In [4]:
# Frequencies allowed = A,Q,M
spec = {'Frequency': 'A,Q,M', 'TableName':'T10101', 'Year':'X'}
tbls_NIPA = show_tables('NIPA', spec)
print(tbls_NIPA[0].index.is_unique)
print(tbls_NIPA[1].index.is_unique)
print(tbls_NIPA[2].index.is_unique)

,,TableName,SeriesCode,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,DataValue,NoteRef
LineNumber,TimePeriod,,,,,,,,
1,1930,T10101,A191RL,Gross domestic product,Fisher Quantity Index,"Percent change, annual rate",0,-8.5,T10101


LineNumber,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,26,27
TimePeriod,,,,,,,,,,,,,,,,,,,,,
1930,-8.5,-5.3,-7.9,-17.2,-5.2,-2.1,-31.9,-22.3,-16.7,-14.4,...,-13.0,-13.0,-15.7,-2.8,10.1,10.7,6.6,15.8,10.0,-11.9


,TableName,SeriesCode,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,NoteRef,1930,1931,1932,...,2020Q1,2020Q2,2020Q3,2020Q4,2021,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1
LineNumber,,,,,,,,,,,,,,,,,,,,,
1,T10101,A191RL,Gross domestic product,Fisher Quantity Index,"Percent change, annual rate",0,T10101,-8.5,-6.4,-12.9,...,-5.1,-31.2,33.8,4.5,5.7,6.3,6.7,2.3,6.9,-1.5


True
True
True


In [5]:
spec = {'Frequency': 'A,Q,M', 'TableName':'U20305', 'Year':'X'}
tbls_NIUnderlyingDetail = show_tables('NIUnderlyingDetail', spec)
print(tbls_NIPA[0].index.is_unique)
print(tbls_NIPA[1].index.is_unique)
print(tbls_NIPA[2].index.is_unique)


,,TableName,SeriesCode,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,DataValue,NoteRef
LineNumber,TimePeriod,,,,,,,,
1,1959,U20305,DPCERC,Personal consumption expenditures (PCE),Current Dollars,Level,6,317130,U20305


LineNumber,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,43,44,45,46
TimePeriod,,,,,,,,,,,,,,,,,,,,,
1959,317130.0,172607.0,44888.0,18789.0,15518.0,6369.0,4212.0,127719.0,61597.0,25366.0,...,4834.0,18762.0,3108.0,19738.0,12107.0,21393.0,2019.0,4864.0,13908.0,9044.0


,TableName,SeriesCode,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,NoteRef,1959,1959M01,1959M02,...,2021M12,2021Q1,2021Q2,2021Q3,2021Q4,2022M01,2022M02,2022M03,2022M04,2022Q1
LineNumber,,,,,,,,,,,,,,,,,,,,,
1,U20305,DPCERC,Personal consumption expenditures (PCE),Current Dollars,Level,6,U20305,317130.0,306091.0,309554.0,...,16242255.0,15005444.0,15681699.0,15964938.0,16314201.0,16575562.0,16676716.0,16906966.0,17059264.0,16719748.0


True
True
True


In [6]:
print("DI")
spec = {'Country':'650,699', 'SeriesId':'27,30', 'DirectionOfInvestment':'Outward', 
        'Year':'2012,2011', 'Classification':'Country', 'GetFootnotes':'Yes'}
tbls_di = show_tables('MNE', spec)
print(tbls_di[0].index.is_unique)
print(tbls_di[1].index.is_unique)
print(tbls_di[2].index.is_unique)

print("AMNE")
spec = {"Year":"2012,2011","Country":"202","Industry":"all","DirectionOfInvestment":"Outward",
        "Classification":"CountryByIndustry","SeriesId":"5,4","NonBankAffiliatesOnly":"0","OwnershipLevel":"0", 'GetFootnotes':'Yes'}

tbls_amne = show_tables('MNE', spec)
print(tbls_amne[0].index.is_unique)
print(tbls_amne[1].index.is_unique)
print(tbls_amne[2].index.is_unique)

DI


,,,,,,,,,SeriesName,ColumnGParent,ColumnParent,ColumnParentCode,TableScale,DataValueUnformatted,DataValue
SeriesID,RowCode,Row,ColumnCode,Column,ColumnGParentCode,Year,TableRowDisplayOrder,TableColumnDisplayOrder,,,,,,,
27,699,Asia and Pacific,0000,Income,0,2012,192.00,1.00,Direct Investment Income Without Current-Cost ...,None,None,0,Millions of Dollars,73749,"73,749"


DataValueUnformatted  \
SeriesID                                                            27   
RowCode                                                            699   
Row                                                   Asia and Pacific   
ColumnCode                                                        0000   
Column                                                          Income   
ColumnGParentCode                                                    0   
Year TableRowDisplayOrder TableColumnDisplayOrder                        
2011 192.00               1.00                                   77158   

                                                                           \
SeriesID                                                        30     27   
RowCode                                                        699    650   
Row                                               Asia and Pacific  China   
ColumnCode                                                    0000   0000   
Column                                                    Position Income   
ColumnGParentCode                                                0      0   
Year TableRowDisplayOrder TableColumnDisplayOrder                           
2011 192.00               1.00                              591657   <NA>   

                                                                  DataValue  \
SeriesID                                                30               27   
RowCode                                                650              699   
Row                                                  China Asia and Pacific   
ColumnCode                                            0000             0000   
Column                                            Position           Income   
ColumnGParentCode                                        0                0   
Year TableRowDisplayOrder TableColumnDisplayOrder                             
2011 192.00               1.00                        <NA>           77,158   

                                                                           \
SeriesID                                                        30     27   
RowCode                                                        699    650   
Row                                               Asia and Pacific  China   
ColumnCode                                                    0000   0000   
Column                                                    Position Income   
ColumnGParentCode                                                0      0   
Year TableRowDisplayOrder TableColumnDisplayOrder                           
2011 192.00               1.00                             591,657   <NA>   

                                                            
SeriesID                                                30  
RowCode                                                650  
Row                                                  China  
ColumnCode                                            0000  
Column                                            Position  
ColumnGParentCode                                        0  
Year TableRowDisplayOrder TableColumnDisplayOrder           
2011 192.00               1.00                        <NA>

,,,,,,SeriesName,ColumnGParent,ColumnParent,ColumnParentCode,TableScale,"(DataValueUnformatted, 2012, 192.00, 1.00)","(DataValueUnformatted, 2011, 192.00, 1.00)","(DataValueUnformatted, 2012, 194.00, 1.00)","(DataValueUnformatted, 2011, 194.00, 1.00)","(DataValue, 2012, 192.00, 1.00)","(DataValue, 2011, 192.00, 1.00)","(DataValue, 2012, 194.00, 1.00)","(DataValue, 2011, 194.00, 1.00)"
SeriesID,RowCode,Row,ColumnCode,Column,ColumnGParentCode,,,,,,,,,,,,,
27,699,Asia and Pacific,0000,Income,0,Direct Investment Income Without Current-Cost ...,None,None,0,Millions of Dollars,73749,77158,<NA>,<NA>,"73,749","77,158",<NA>,<NA>


True
True
True
AMNE


,,,,,,,,,SeriesName,ColumnGParent,ColumnParent,ColumnParentCode,TableScale,DataValueUnformatted,DataValue
SeriesID,RowCode,Row,ColumnCode,Column,ColumnGParentCode,Year,TableRowDisplayOrder,TableColumnDisplayOrder,,,,,,,
5,202,Brazil,0000,All Industries Total,0,2012,1.29,1.00,Net Income,None,None,0,Millions of Dollars,10407,"10,407"


DataValueUnformatted  \
SeriesID                                                             5   
RowCode                                                            202   
Row                                                             Brazil   
ColumnCode                                                        0000   
Column                                            All Industries Total   
ColumnGParentCode                                                    0   
Year TableRowDisplayOrder TableColumnDisplayOrder                        
2011 1.29                 1.00                                   13084   

                                                                               \
SeriesID                                                             4      5   
RowCode                                                            202    202   
Row                                                             Brazil Brazil   
ColumnCode                                                        0000   2100   
Column                                            All Industries Total Mining   
ColumnGParentCode                                                    0      0   
Year TableRowDisplayOrder TableColumnDisplayOrder                               
2011 1.29                 1.00                                  202988   <NA>   

                                                                              \
SeriesID                                               4                       
RowCode                                              202                       
Row                                               Brazil                       
ColumnCode                                          2100                3000   
Column                                            Mining Total Manufacturing   
ColumnGParentCode                                      0                   0   
Year TableRowDisplayOrder TableColumnDisplayOrder                              
2011 1.29                 1.00                      <NA>                <NA>   

                                                                              \
SeriesID                                                            5      4   
RowCode                                                           202    202   
Row                                                            Brazil Brazil   
ColumnCode                                                       3000   3110   
Column                                            Total Manufacturing   Food   
ColumnGParentCode                                                   0   3000   
Year TableRowDisplayOrder TableColumnDisplayOrder                              
2011 1.29                 1.00                                   <NA>   <NA>   

                                                                              \
SeriesID                                               5                   4   
RowCode                                              202                 202   
Row                                               Brazil              Brazil   
ColumnCode                                          3110      3250      3250   
Column                                              Food Chemicals Chemicals   
ColumnGParentCode                                   3000      3000      3000   
Year TableRowDisplayOrder TableColumnDisplayOrder                              
2011 1.29                 1.00                      <NA>      <NA>      <NA>   

                                                   ...    DataValue  \
SeriesID                                           ...            4   
RowCode                                            ...          202   
Row                                                ...       Brazil   
ColumnCode                                         ...         4400   
Column                                             ... Retail Trade   
ColumnGParentCode                                  ...          

,,,,,,SeriesName,ColumnGParent,ColumnParent,ColumnParentCode,TableScale,"(DataValueUnformatted, 2012, 1.29, 1.00)","(DataValueUnformatted, 2011, 1.29, 1.00)","(DataValueUnformatted, 2011, 1.29, 2.00)","(DataValueUnformatted, 2012, 1.29, 2.00)","(DataValueUnformatted, 2011, 1.29, 3.00)",...,"(DataValue, 2011, 1.29, 12.00)","(DataValue, 2012, 1.29, 12.00)","(DataValue, 2012, 1.29, 13.00)","(DataValue, 2011, 1.29, 13.00)","(DataValue, 2011, 1.29, 14.00)","(DataValue, 2012, 1.29, 14.00)","(DataValue, 2011, 1.29, 15.00)","(DataValue, 2012, 1.29, 15.00)","(DataValue, 2011, 1.29, 16.00)","(DataValue, 2012, 1.29, 16.00)"
SeriesID,RowCode,Row,ColumnCode,Column,ColumnGParentCode,,,,,,,,,,,,,,,,,,,,,
5,202,Brazil,0000,All Industries Total,0,Net Income,None,None,0,Millions of Dollars,10407,13084,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


True
True
True


In [7]:
spec = {'TableName':'FAAt201', 'Year':'X'}
tbls_FixedAssets = show_tables('FixedAssets', spec)
print(tbls_FixedAssets[0].index.is_unique)
print(tbls_FixedAssets[1].index.is_unique)
print(tbls_FixedAssets[2].index.is_unique)

,,,TableName,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,DataValue,NoteRef
SeriesCode,LineNumber,TimePeriod,,,,,,,
k1ptotl1es00,1,1925,FAAt201,Private fixed assets,Current Dollars,Level,9,223.4,FAAt201


SeriesCode,k1ptotl1es00,k1ptotl1eq00,k1ntotl1eq00,k1ntotl1ep00,k1ntotl1ep11,k1ntotl1ep20,k1ntotl1ep33,k1ntotl1ep36,k1ntotl1ep31,k1ntotl1ep12,...,k1ntotl1rdon,k1ntotl1rdnp,k1ntotl1rd91,k1ntotl1rd92,k1ntotl1ae00,k1ntotl1ae10,k1ntotl1ae20,k1ntotl1ae30,k1ntotl1ae40,k1ntotl1ae50
LineNumber,1,2,3,4,5,6,7,8,9,10,...,94,95,96,97,98,99,100,101,102,103
TimePeriod,,,,,,,,,,,,,,,,,,,,,
1925,223.4,31.4,31.1,1.4,0.0,0.7,0.0,0.1,0.0,0.5,...,0.1,0.1,0.1,0.0,1.4,0.6,0.0,0.5,0.1,0.2


,,TableName,LineDescription,METRIC_NAME,CL_UNIT,UNIT_MULT,NoteRef,1925,1926,1927,1928,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
SeriesCode,LineNumber,,,,,,,,,,,,,,,,,,,,,
k1ptotl1es00,1,FAAt201,Private fixed assets,Current Dollars,Level,9,FAAt201,223.4,231.6,236.9,248.6,...,35444.0,36569.3,38567.6,40346.1,41443.7,43342.2,45154.9,47599.6,49540.9,51719.6


True
True
True


In [8]:
print("One indicator")
spec = {'Indicator':"BalGds","AreaOrCountry":"China,Brazil","Frequency":"A,QSA,QNSA", "Year":"2011,2012"}
tbls_ITAa = show_tables('ITA', spec) # TODO: Failed for itable_style=False
print(tbls_ITAa[0].index.is_unique)
print(tbls_ITAa[1].index.is_unique)
print(tbls_ITAa[2].index.is_unique)

print("One Area")
spec = {'Indicator':"BalGds,PfInvAssets","AreaOrCountry":"China","Frequency":"A,QSA,QNSA", "Year":"2011,2012"}
tbls_ITAb = show_tables('ITA', spec) # TODO: Failed for itable_style=False
print(tbls_ITAb[0].index.is_unique)
print(tbls_ITAb[1].index.is_unique)
print(tbls_ITAb[2].index.is_unique)

One indicator


,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
Indicator,AreaOrCountry,Frequency,TimePeriod,,,,,,,
BalGds,Brazil,A,2011,2011,TSI_ItaBalGdsBrazil_A,Balance on goods; Brazil; annual,USD,6,11423,


DataValue                             NoteRef                    \
Indicator        BalGds                              BalGds                     
AreaOrCountry    Brazil              China           Brazil             China   
Frequency             A QNSA QSA         A QNSA QSA       A  QNSA   QSA     A   
TimePeriod                                                                      
2011            11423.0  NaN NaN -295187.0  NaN NaN          <NA>  <NA>         

                           
Indicator                  
AreaOrCountry              
Frequency      QNSA   QSA  
TimePeriod                 
2011           <NA>  <NA>

,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 2011)","(DataValue, 2011Q1)","(DataValue, 2011Q2)","(DataValue, 2011Q3)","(DataValue, 2011Q4)","(DataValue, 2012)",...,"(NoteRef, 2011)","(NoteRef, 2011Q1)","(NoteRef, 2011Q2)","(NoteRef, 2011Q3)","(NoteRef, 2011Q4)","(NoteRef, 2012)","(NoteRef, 2012Q1)","(NoteRef, 2012Q2)","(NoteRef, 2012Q3)","(NoteRef, 2012Q4)"
Indicator,AreaOrCountry,Frequency,,,,,,,,,,,,,,,,,,,,,
BalGds,Brazil,A,TSI_ItaBalGdsBrazil_A,Balance on goods; Brazil; annual,USD,6,11423.0,NaN,NaN,NaN,NaN,11913.0,...,,<NA>,<NA>,<NA>,<NA>,,<NA>,<NA>,<NA>,<NA>


True
True
True
One Area


,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
Indicator,AreaOrCountry,Frequency,TimePeriod,,,,,,,
BalGds,China,A,2011,2011,TSI_ItaBalGdsChina_A,Balance on goods; China; annual,USD,6,-295187,


DataValue                           NoteRef              \
Indicator        BalGds          PfInvAssets       BalGds               
AreaOrCountry     China                China        China               
Frequency             A QNSA QSA           A QNSA       A  QNSA   QSA   
TimePeriod                                                              
2011          -295187.0  NaN NaN     -9568.0  NaN          <NA>  <NA>   

                                 
Indicator     PfInvAssets        
AreaOrCountry       China        
Frequency               A  QNSA  
TimePeriod                       
2011                       <NA>

,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 2011)","(DataValue, 2011Q1)","(DataValue, 2011Q2)","(DataValue, 2011Q3)","(DataValue, 2011Q4)","(DataValue, 2012)",...,"(NoteRef, 2011)","(NoteRef, 2011Q1)","(NoteRef, 2011Q2)","(NoteRef, 2011Q3)","(NoteRef, 2011Q4)","(NoteRef, 2012)","(NoteRef, 2012Q1)","(NoteRef, 2012Q2)","(NoteRef, 2012Q3)","(NoteRef, 2012Q4)"
Indicator,AreaOrCountry,Frequency,,,,,,,,,,,,,,,,,,,,,
BalGds,China,A,TSI_ItaBalGdsChina_A,Balance on goods; China; annual,USD,6,-295187.0,NaN,NaN,NaN,NaN,-314937.0,...,,<NA>,<NA>,<NA>,<NA>,,<NA>,<NA>,<NA>,<NA>


True
True
True


In [9]:
print("Specific TypeOfInvestment")
spec = {"TypeOfInvestment":"FinAssetsExclFinDeriv","Component":"ALL","Frequency":"ALL","Year":"ALL"}
tbls_IIPa = show_tables("IIP", spec)
print(tbls_IIPa[0].index.is_unique)
#print(tbls_IIPa[1].duplicated(['Component','Frequency']).sum()==0) # Note, could've combined A and 'QNSA' into one row (and drop freq column), but wouldn't work if allowed 'QSA'
print(tbls_IIPa[1].index.is_unique)
print(tbls_IIPa[2].index.is_unique)
print(tbls_IIPa[0].reset_index()["Frequency"].unique())  # ['A', 'QNSA']

print("Specific Year")
spec = {"TypeOfInvestment":"ALL","Component":"ALL","Frequency":"ALL","Year":"2015"}
tbls_IIPb = show_tables("IIP", spec)
print(tbls_IIPb[0].index.is_unique)
print(tbls_IIPb[1].index.is_unique)
print(tbls_IIPb[2].index.is_unique)

Specific TypeOfInvestment


,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
Frequency,TypeOfInvestment,Component,TimePeriod,,,,,,,
A,FinAssetsExclFinDeriv,ChgPos,2003,2003,TSI_IipFinAssetsExclFinDerivChgPos_A,U.S. assets excluding financial derivatives (C...,USD,6,1555787,


DataValue                                  \
Frequency                            A                                   
TypeOfInvestment FinAssetsExclFinDeriv                                   
Component                       ChgPos ChgPosNie ChgPosOth ChgPosPrice   
TimePeriod                                                               
1976                               NaN       NaN       NaN         NaN   

                                                                          \
Frequency                                                           QNSA   
TypeOfInvestment                                   FinAssetsExclFinDeriv   
Component        ChgPosTrans ChgPosXRate       Pos                   Pos   
TimePeriod                                                                 
1976                     NaN         NaN  371424.0                   NaN   

                               NoteRef                                  \
Frequency                            A                                   
TypeOfInvestment FinAssetsExclFinDeriv                                   
Component                       ChgPos ChgPosNie ChgPosOth ChgPosPrice   
TimePeriod                                                               
1976                              <NA>      <NA>      <NA>        <NA>   

                                                                    
Frequency                                                     QNSA  
TypeOfInvestment                             FinAssetsExclFinDeriv  
Component        ChgPosTrans ChgPosXRate Pos                   Pos  
TimePeriod                                                          
1976                    <NA>        <NA>                      <NA>

c:\Users\Brian\miniconda3\envs\BEA_API\lib\site-packages\pandas\core\reshape\merge.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  return op.get_result()


,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 1976)","(DataValue, 1977)","(DataValue, 1978)","(DataValue, 1979)","(DataValue, 1980)","(DataValue, 1981)",...,"(NoteRef, 2020)","(NoteRef, 2020Q1)","(NoteRef, 2020Q2)","(NoteRef, 2020Q3)","(NoteRef, 2020Q4)","(NoteRef, 2021)","(NoteRef, 2021Q1)","(NoteRef, 2021Q2)","(NoteRef, 2021Q3)","(NoteRef, 2021Q4)"
Frequency,TypeOfInvestment,Component,,,,,,,,,,,,,,,,,,,,,
A,FinAssetsExclFinDeriv,ChgPos,TSI_IipFinAssetsExclFinDerivChgPos_A,U.S. assets excluding financial derivatives (C...,USD,6,NaN,NaN,NaN,NaN,NaN,NaN,...,,<NA>,<NA>,<NA>,<NA>,,<NA>,<NA>,<NA>,<NA>


True
True
True
['A' 'QNSA']
Specific Year


,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
Frequency,TypeOfInvestment,Component,TimePeriod,,,,,,,
A,CurrAndDepAssets,ChgPos,2015,2015,TSI_IipCurrAndDepAssetsChgPos_A,U.S. assets; other investment; currency and de...,USD,6,-174211.0,


DataValue                                              \
Frequency                       A                                               
TypeOfInvestment CurrAndDepAssets                                               
Component                  ChgPos ChgPosNie ChgPosOth ChgPosPrice ChgPosTrans   
TimePeriod                                                                      
2015                    -174211.0   47838.0   17261.0         NaN   -191472.0   

                                                                          \
Frequency                                           QNSA               A   
TypeOfInvestment                        CurrAndDepAssets CurrAndDepLiabs   
Component        ChgPosXRate        Pos              Pos          ChgPos   
TimePeriod                                                                 
2015                -30577.0  1632883.0              NaN         55769.0   

                            ...                 NoteRef  \
Frequency                   ...                       A   
TypeOfInvestment            ... TreasBillsAndCertsLiabs   
Component        ChgPosNie  ...                     Pos   
TimePeriod                  ...                           
2015               31500.0  ...                           

                                                                            \
Frequency                           QNSA                       A             
TypeOfInvestment TreasBillsAndCertsLiabs TreasBondsAndNotesLiabs             
Component                            Pos                  ChgPos ChgPosNie   
TimePeriod                                                                   
2015                                <NA>                          IIP1.3:2   

                                                                        \
Frequency                                                                
TypeOfInvestment                                                         
Component        ChgPosOth ChgPosPrice ChgPosTrans     ChgPosXRate Pos   
TimePeriod                                                               
2015                                                .....,IIP1.3:1       

                                          
Frequency                           QNSA  
TypeOfInvestment TreasBondsAndNotesLiabs  
Component                            Pos  
TimePeriod                                
2015                                <NA>  

[1 rows x 1128 columns]

,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 2015)","(DataValue, 2015Q1)","(DataValue, 2015Q2)","(DataValue, 2015Q3)","(DataValue, 2015Q4)","(NoteRef, 2015)","(NoteRef, 2015Q1)","(NoteRef, 2015Q2)","(NoteRef, 2015Q3)","(NoteRef, 2015Q4)"
Frequency,TypeOfInvestment,Component,,,,,,,,,,,,,,
A,CurrAndDepAssets,ChgPos,TSI_IipCurrAndDepAssetsChgPos_A,U.S. assets; other investment; currency and de...,USD,6,-174211.0,NaN,NaN,NaN,NaN,,<NA>,<NA>,<NA>,<NA>


True
True
True


In [10]:
spec = {"Year":"2010,2011,2012,2013","tableID":"56"} # Note: due to api error, only as for one table at a time
tbls_InputOutput = show_tables("InputOutput", spec)

print(tbls_InputOutput[0].index.is_unique)
print(tbls_InputOutput[1].index.is_unique)
print(tbls_InputOutput[2].index.is_unique)

,,,,RowDescr,RowType,ColDescr,ColType,DataValue,NoteRef
TableID,RowCode,ColCode,Year,,,,,,
56,,Other,2010,Total industry output requirement,Industry,Noncomparable imports and rest-of-the-world ad...,Commodity,1.706205,56


TableID        56                                                             \
RowCode                                                                        
ColCode     Other      Used        G        81         7         6      PROF   
Year                                                                           
2010     1.706205  1.989144  1.70626  1.629452  1.766178  1.678389  1.594069   

TableID                                ...                               \
RowCode                                ...      PROF                      
ColCode      FIRE        51      48TW  ...      Used       21     Other   
Year                                   ...                                
2010     1.595031  1.727656  1.921793  ...  0.259319  0.11848  0.137893   

TableID                                                                       
RowCode                                                                       
ColCode       22        23        51      48TW      44RT        42       31G  
Year                                                                          
2010     0.12492  0.110734  0.172346  0.117592  0.156418  0.172228  0.131669  

[1 rows x 272 columns]

,,,RowDescr,RowType,ColType,ColDescr,NoteRef,2010,2011,2012,2013
TableID,RowCode,ColCode,,,,,,,,,
56,,Other,Total industry output requirement,Industry,Commodity,Noncomparable imports and rest-of-the-world ad...,56,1.706205,1.725841,1.718243,1.699646


True
True
True


In [11]:
print("One Country")
spec = {"AreaOrCountry":"Germany","Year":"1999,2000"}
tbls_IntlServTrade_a = show_tables("IntlServTrade", spec)
print(tbls_IntlServTrade_a[0].index.is_unique)
## TODO: The wide cols_as_vars table doubles each row and puts blank data in one (w/ a non-empty NoteRef) and the other has data but no note ref. That's why noteref is required in the index
#print(tbls_IntlServTrade_a[1].duplicated(['TypeOfService','TradeDirection','Affiliation', 'NoteRef']).sum()) # SHouldn't have to include noteref
display(tbls_IntlServTrade_a[1].index.is_unique)
print(tbls_IntlServTrade_a[2].index.is_unique)


print("One type of service")
spec = {"TypeOfService":"Telecom", "AreaOrCountry":"Germany,France","Year":"1999,2000"}
tbls_IntlServTrade_b = show_tables("IntlServTrade", spec) # TODO: Failed on itable_style=False table
print(tbls_IntlServTrade_b[0].index.is_unique)
print(tbls_IntlServTrade_b[1].index.is_unique)
print(tbls_IntlServTrade_b[2].index.is_unique)

One Country


,,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
TypeOfService,TradeDirection,Affiliation,AreaOrCountry,TimePeriod,,,,,,,
AccountAuditBookkeep,Exports,AllAffiliations,Germany,1999,1999,TSI_IstAccountAuditBookkeepExpGermanyAllAff_A,"Exports of accounting, auditing, and bookkeepi...",USD,6,NaN,n.a.


DataValue                                  \
TypeOfService  AccountAuditBookkeep                     Advertising   
TradeDirection              Exports         Imports         Exports   
Affiliation         AllAffiliations AllAffiliations AllAffiliations   
AreaOrCountry               Germany         Germany         Germany   
TimePeriod                                                            
1999                            NaN             NaN             NaN   

                                                                      \
TypeOfService                  AdvertisingAndRelated                   
TradeDirection         Imports               Exports         Imports   
Affiliation    AllAffiliations       AllAffiliations AllAffiliations   
AreaOrCountry          Germany               Germany         Germany   
TimePeriod                                                             
1999                       NaN                   NaN             NaN   

                                                                  \
TypeOfService     AgForAndFish                 AllTypesOfService   
TradeDirection         Exports         Imports           Balance   
Affiliation    AllAffiliations AllAffiliations   AllAffiliations   
AreaOrCountry          Germany         Germany           Germany   
TimePeriod                                                         
1999                       NaN             NaN            2212.0   

                                ...         NoteRef                  \
TypeOfService                   ...  TravelPersonal                   
TradeDirection         Exports  ...         Exports         Imports   
Affiliation    AllAffiliations  ... AllAffiliations AllAffiliations   
AreaOrCountry          Germany  ...         Germany         Germany   
TimePeriod                      ...                                   
1999                   16484.0  ...                                   

                                                                      \
TypeOfService  TravelPersonalOth                 TravelShortTermWork   
TradeDirection           Exports         Imports             Exports   
Affiliation      AllAffiliations AllAffiliations     AllAffiliations   
AreaOrCountry            Germany         Germany             Germany   
TimePeriod                                                             
1999                                                                   

                                                                   \
TypeOfService                  WasteTreatAndDePol                   
TradeDirection         Imports            Exports         Imports   
Affiliation    AllAffiliations    AllAffiliations AllAffiliations   
AreaOrCountry          Germany            Germany         Germany   
TimePeriod                                                          
1999                                         n.a.            n.a.   

                                                              
TypeOfService  WasteTreatAndDePolAgAndMining                  
TradeDirection                       Exports         Imports  
Affiliation                  AllAffiliations AllAffiliations  
AreaOrCountry                        Germany         Germany  
TimePeriod                                                    
1999                                    n.a.            n.a.  

[1 rows x 438 columns]

,,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 1999)","(DataValue, 2000)","(NoteRef, 1999)","(NoteRef, 2000)"
TypeOfService,TradeDirection,Affiliation,AreaOrCountry,,,,,,,,
AccountAuditBookkeep,Exports,AllAffiliations,Germany,TSI_IstAccountAuditBookkeepExpGermanyAllAff_A,"Exports of accounting, auditing, and bookkeepi...",USD,6,NaN,NaN,n.a.,n.a.


True


...

True
One type of service


,,,,,Year,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,DataValue,NoteRef
TypeOfService,TradeDirection,Affiliation,AreaOrCountry,TimePeriod,,,,,,,
Telecom,Exports,AllAffiliations,France,1999,1999,TSI_IstTelecomExpFranceAllAff_A,Exports of telecommunications services; France,USD,6,NaN,n.a.


DataValue                                  \
TypeOfService          Telecom                                   
TradeDirection         Exports                 Imports           
Affiliation    AllAffiliations         AllAffiliations           
AreaOrCountry           France Germany          France Germany   
TimePeriod                                                       
1999                       NaN     NaN             NaN     NaN   

                       NoteRef                                  
TypeOfService          Telecom                                  
TradeDirection         Exports                 Imports          
Affiliation    AllAffiliations         AllAffiliations          
AreaOrCountry           France Germany          France Germany  
TimePeriod                                                      
1999                      n.a.    n.a.            n.a.    n.a.

,,,,TimeSeriesId,TimeSeriesDescription,CL_UNIT,UNIT_MULT,"(DataValue, 1999)","(DataValue, 2000)","(NoteRef, 1999)","(NoteRef, 2000)"
TypeOfService,TradeDirection,Affiliation,AreaOrCountry,,,,,,,,
Telecom,Exports,AllAffiliations,France,TSI_IstTelecomExpFranceAllAff_A,Exports of telecommunications services; France,USD,6,NaN,NaN,n.a.,n.a.


True
True
True


In [12]:
spec = {'Year':'2012,2011','Industry':'ALL','tableID':'1, 2', "Frequency":"A,Q"}
tbls_GDPbyIndustry = show_tables('GDPbyIndustry', spec)
print(tbls_GDPbyIndustry[0].index.is_unique)
print(tbls_GDPbyIndustry[1].index.is_unique)
print(tbls_GDPbyIndustry[2].index.is_unique)

,,,,,,DataValue,NoteRef
TableID,Industry,IndustrYDescription,Frequency,Year,Quarter,,
1,11,"Agriculture, forestry, fishing, and hunting",A,2011,2011,179.2,1


TableID                                                          1         \
Industry                                                        11  111CA   
IndustrYDescription    Agriculture, forestry, fishing, and hunting  Farms   
Frequency Year Quarter                                                      
A         2011 2011                                          179.2  150.4   

TableID                                                                  \
Industry                                                   113FF     21   
IndustrYDescription    Forestry, fishing, and related activities Mining   
Frequency Year Quarter                                                    
A         2011 2011                                         28.8  357.8   

TableID                                                                   \
Industry                                  211                        212   
IndustrYDescription    Oil and gas extraction Mining, except oil and gas   
Frequency Year Quarter                                                     
A         2011 2011                     219.0                       76.7   

TableID                                                                      \
Industry                                         213        22           23   
IndustrYDescription    Support activities for mining Utilities Construction   
Frequency Year Quarter                                                        
A         2011 2011                             62.0     288.3        525.6   

TableID                                                        ...  \
Industry                                                311FT  ...   
IndustrYDescription    Food and beverage and tobacco products  ...   
Frequency Year Quarter                                         ...   
A         2011 2011                                     209.8  ...   

TableID                                                        \
Industry                           GSL                   GSLE   
IndustrYDescription    State and local Government enterprises   
Frequency Year Quarter                                          
A         2011 2011             1424.8                  111.7   

TableID                                            \
Industry                             GSLG      HS   
IndustrYDescription    General government Housing   
Frequency Year Quarter                              
A         2011 2011                1313.1  1523.9   

TableID                                                                                        \
Industry                                                                                  ICT   
IndustrYDescription    Information-communications-technology-producing industries<sup>3</sup>   
Frequency Year Quarter                                                                          
A         2011 2011                                                 977.7                       

TableID                                   \
Industry                             ORE   
IndustrYDescription    Other real estate   
Frequency Year Quarter                     
A         2011 2011                360.0   

TableID                                                                \
Industry                                                        PGOOD   
IndustrYDescription    Private goods-producing industries<sup>1</sup>   
Frequency Year Quarter                                                  
A         2011 2011                                            2936.1   

TableID                                                    \
Industry                                             PROF   
IndustrYDescription    Professional and business services   
Frequency Year Quarter                                      
A         2011 2011                                1860.0   

TableID                                                                   \
Industry                                                       

,,,NoteRef,"(A, 2011, 2011)","(A, 2012, 2012)","(Q, 2011, I)","(Q, 2011, II)","(Q, 2011, III)","(Q, 2011, IV)","(Q, 2012, I)","(Q, 2012, II)","(Q, 2012, III)","(Q, 2012, IV)"
TableID,Industry,IndustrYDescription,,,,,,,,,,,
1,11,"Agriculture, forestry, fishing, and hunting",1,179.2,178.7,181.5,173.4,181.4,180.4,181.6,177.8,174.9,180.3


True
True
True


In [13]:
spec = {"Year":"2013,2012","Industry":"ALL","tableID":"210,211","Frequency":"A,Q"}
tbls_underlyingGDPbyIndustry = show_tables("underlyingGDPbyIndustry", spec)
print(tbls_underlyingGDPbyIndustry[0].index.is_unique)
print(tbls_underlyingGDPbyIndustry[1].index.is_unique)
print(tbls_underlyingGDPbyIndustry[2].index.is_unique)

underlyinggdpbyindustry currenty only returns Annual results (even if you ask for additional frequencies).


,,,Frequency,IndustrYDescription,DataValue,NoteRef
TableID,Industry,Year,,,,
210,11,2012,A,"Agriculture, forestry, fishing, and hunting",178.7,210


TableID     210                                                            \
Industry     11   111  111CA   112 113FF     21    211   212   213     22   
Year                                                                        
2012      178.7  88.1  148.0  59.9  30.7  360.5  224.9  67.0  68.6  280.7   

TableID   ...   211                                                   
Industry  ... GSLGO   HS  HSO  HST  ICT  ORE PGOOD  PROF PSERV   PVT  
Year      ...                                                         
2012      ...   3.1  9.5  7.1  2.4  6.1  2.4  18.6  12.1  68.1  86.7  

[1 rows x 382 columns]

,,Frequency,IndustrYDescription,NoteRef,2012,2013
TableID,Industry,,,,,
210,11,A,"Agriculture, forestry, fishing, and hunting",210,178.7,214.3


True
True
True


In [14]:
print("1 line code")
spec = {"TableName":"CAINC1","LineCode":"2","Year":"2012,2013","GeoFips":"09003,53033"}
tbls_Regional_a = show_tables("Regional", spec) # TODO: Failed for itable_style=False
print(tbls_Regional_a[0].index.is_unique)
print(tbls_Regional_a[1].index.is_unique)
print(tbls_Regional_a[2].index.is_unique)

print("1 GeoFips")
spec = {"TableName":"CAINC1","LineCode":"All","Year":"2012,2013","GeoFips":"09001"}
tbls_Regional_b = show_tables("Regional", spec) # TODO: Failed for itable_style=False
print(tbls_Regional_b[0].index.is_unique)
print(tbls_Regional_b[1].index.is_unique)
print(tbls_Regional_b[2].index.is_unique)

1 line code


,,,GeoName,CL_UNIT,UNIT_MULT,DataValue
Code,GeoFips,TimePeriod,,,,
CAINC1-2,09003,2012,"Hartford, CT",Number of persons,0,897795


Code       CAINC1-2         
GeoFips       09003    53033
TimePeriod                  
2012         897795  2011708

,,GeoName,CL_UNIT,UNIT_MULT,2012,2013
Code,GeoFips,,,,,
CAINC1-2,09003,"Hartford, CT",Number of persons,0,897795,897837


True
True
True
1 GeoFips


,,,GeoName,Description,CL_UNIT,UNIT_MULT,DataValue
Code,GeoFips,TimePeriod,,,,,
CAINC1-1,09001,2012,"Fairfield, CT",Personal income (thousands of dollars),Thousands of dollars,3,96709567


Code,CAINC1-1,CAINC1-3,CAINC1-2
GeoFips,09001,09001,09001
TimePeriod,,,
2012,96709567,103400,935293


,,GeoName,CL_UNIT,UNIT_MULT,Description,2012,2013
Code,GeoFips,,,,,,
CAINC1-1,09001,"Fairfield, CT",Thousands of dollars,3,Personal income (thousands of dollars),96709567,91249002


True
True
True
